# A Look at Wildlife Diseases in Utah

This notebook analyses wildlife disease data from Utah DWR captures in the 2021-2022 fall/winter capture season. Please see [the github repo](https://github.com/epistemetrica/utah_dwr_python) for more info. 

## Project Description

Each observation in our dataset includes big game animal species, capture date and location, sex, pregnancy status, and the results of various disease-related lab tests.

### Key tasks:
1. Prepare the data for analysis 
2. Explore and visualize the dataset
3. Analyze the data to understand:
    - the prevelance of key diseases among wildlife in Utah
    - whether this data allows us to accurately predict disease states for new captures




In [1]:
#import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

## Data Processing

Lab results and DWR capture data were loaded into a SQLite db via the wildlife_ETL.py pipeline. 

In [2]:
#load db
engine = create_engine('sqlite:///data/wildlife.db')

#load dfs
bison_df = pd.read_sql_table(con=engine, table_name='bison_table')
deer_df = pd.read_sql_table(con=engine, table_name='deer_table')
elk_df = pd.read_sql_table(con=engine, table_name='elk_table')
moose_df = pd.read_sql_table(con=engine, table_name='moose_table')
pronghorn_df = pd.read_sql_table(con=engine, table_name='pronghorn_table')
sheep_goat_df = pd.read_sql_table(con=engine, table_name='sheep_goat_table')

Let's take a look at the data:

In [3]:
display(bison_df.head())
bison_df.info()

,sample_id,archive_id,species,sex,capture_date,capture_unit,bvd_type1_result,bvd_type2_result,ehdv_result,preg_val,preg_result,bluetongue_result
0,BCB1101,BIS21-018,Bison,Female,2022-02-27,Book Cliffs,Negative,Negative,Positive,0.091,Not pregnant,Detected
1,BCB1102,BIS21-019,Bison,Female,2022-02-26,Book Cliffs,Negative,Negative,Negative,0.749,Pregnant,Not detected
2,BCB1103,BIS21-020,Bison,Female,2022-02-27,Book Cliffs,Negative,Negative,Negative,0.091,Not pregnant,Not detected
3,BCB1104,BIS21-021,Bison,Female,2022-02-26,Book Cliffs,Negative,Negative,Negative,0.718,Pregnant,Not detected
4,BCB1105,BIS21-022,Bison,Male,2022-02-26,Book Cliffs,Negative,Negative,Negative,0.089,Not pregnant,Not detected


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   sample_id          35 non-null     object        
 1   archive_id         15 non-null     object        
 2   species            33 non-null     object        
 3   sex                33 non-null     object        
 4   capture_date       33 non-null     datetime64[ns]
 5   capture_unit       33 non-null     object        
 6   bvd_type1_result   34 non-null     object        
 7   bvd_type2_result   34 non-null     object        
 8   ehdv_result        34 non-null     object        
 9   preg_val           34 non-null     float64       
 10  preg_result        34 non-null     object        
 11  bluetongue_result  34 non-null     object        
dtypes: datetime64[ns](1), float64(1), object(10)
memory usage: 3.4+ KB


In [4]:
display(deer_df.head())
deer_df.info()

,sample_id,collar_id,species,sex,capture_date,capture_unit,adenovirus_result,ehdv_result,bluetongue_result
0,BC1101,50713.0,Mule Deer,Female,2021-12-11,Book Cliffs,>=1:4096,Negative,Not detected
1,BC1102,50716.0,Mule Deer,Female,2021-12-11,Book Cliffs,1:16,Negative,Not detected
2,BC1103,50714.0,Mule Deer,Female,2021-12-11,Book Cliffs,1:8,Positive,Detected
3,BC1104,50720.0,Mule Deer,Female,2021-12-11,Book Cliffs,>=1:4096,Negative,Not detected
4,BC1105,50722.0,Mule Deer,Male,2021-12-11,Book Cliffs,>=1:4096,Negative,Not detected


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 465 entries, 0 to 464
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   sample_id          465 non-null    object        
 1   collar_id          459 non-null    float64       
 2   species            459 non-null    object        
 3   sex                459 non-null    object        
 4   capture_date       459 non-null    datetime64[ns]
 5   capture_unit       459 non-null    object        
 6   adenovirus_result  462 non-null    object        
 7   ehdv_result        462 non-null    object        
 8   bluetongue_result  461 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 32.8+ KB


In [6]:
display(elk_df.head())
elk_df.info()

,sample_id,archive_id,collar_id,species,sex,capture_date,capture_unit,staging_area,age,comments,ehdv_type1_result,ehdv_type1_val,ehdv_type2_result,ehdv_type2_val,ehdv_type6_result,ehdv_type6_val,bvd_result,preg_val,preg_result,bluetongue_result
0,BCE1132,EL21-137,51628.0,Elk,Female,2022-03-07,Book Cliffs,Bryson Canyon,23.5,None,Negative,<20,Positve,>1280,Negative,<20,Negative,0.660,Pregnant,Detected
1,BCE1134,EL21-138,46687.0,Elk,Female,2022-03-07,Book Cliffs,Bryson Canyon,19.5,None,Negative,<20,Negative,<20,Negative,<20,Negative,0.649,Pregnant,Detected
2,BCE1135,EL21-139,46752.0,Elk,Female,2022-03-07,Book Cliffs,Bryson Canyon,10.5,None,Negative,<20,Positive,40,Negative,<20,Negative,0.087,Open,Not detected
3,BCE1136,EL21-140,51622.0,Elk,Female,2022-03-07,Book Cliffs,Bryson Canyon,13.5,None,Negative,<20,Negative,<20,Negative,<20,Negative,0.621,Pregnant,Not detected
4,BCE1138,EL21-141,46700.0,Elk,Female,2022-03-07,Book Cliffs,Bryson Canyon,22.5,None,Negative,<20,Negative,<20,Negative,<20,Negative,0.612,Pregnant,Not detected


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149 entries, 0 to 148
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   sample_id          149 non-null    object        
 1   archive_id         46 non-null     object        
 2   collar_id          147 non-null    float64       
 3   species            147 non-null    object        
 4   sex                147 non-null    object        
 5   capture_date       147 non-null    datetime64[ns]
 6   capture_unit       147 non-null    object        
 7   staging_area       136 non-null    object        
 8   age                146 non-null    float64       
 9   comments           9 non-null      object        
 10  ehdv_type1_result  146 non-null    object        
 11  ehdv_type1_val     146 non-null    object        
 12  ehdv_type2_result  146 non-null    object        
 13  ehdv_type2_val     146 non-null    object        
 14  ehdv_type6

In [7]:
display(moose_df.head())
moose_df.info()

,sample_id,collar_id,species,sex,capture_date,capture_unit,staging_area,preg_val,preg_result,bluetongue_result
0,PAR1111,50888,Moose,Female,2021-12-19,Parley's Summit,Little Dell Reservoir,0.105,Open,Not detected
1,PAR1112,50891,Moose,Female,2021-12-19,Parley's Summit,Little Dell Reservoir,1.165,Pregnant,Not detected
2,PAR1113,50892,Moose,Female,2021-12-19,Parley's Summit,Little Dell Reservoir,1.180,Pregnant,Not detected
3,PAR1114,50890,Moose,Female,2021-12-19,Parley's Summit,Little Dell Reservoir,1.202,Pregnant,Not detected
4,PAR1115,50889,Moose,Female,2021-12-19,Parley's Summit,Little Dell Reservoir,1.177,Pregnant,Not detected


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   sample_id          5 non-null      object        
 1   collar_id          5 non-null      int64         
 2   species            5 non-null      object        
 3   sex                5 non-null      object        
 4   capture_date       5 non-null      datetime64[ns]
 5   capture_unit       5 non-null      object        
 6   staging_area       5 non-null      object        
 7   preg_val           5 non-null      float64       
 8   preg_result        5 non-null      object        
 9   bluetongue_result  5 non-null      object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(7)
memory usage: 532.0+ bytes


In [8]:
display(pronghorn_df.head())
pronghorn_df.info()

,sample_id,archive_id,collar_id,species,sex,capture_date,capture_unit,staging_area,bluetongue_result
0,BCP1101,PH21-001,50770.0,Pronghorn,Female,2021-12-01,Book Cliffs,Danish Flat,Detected
1,BCP1102,PH21-002,50776.0,Pronghorn,Female,2021-12-01,Book Cliffs,Danish Flat,None
2,BCP1103,PH21-003,50769.0,Pronghorn,Female,2021-12-01,Book Cliffs,Danish Flat,Detected
3,BCP1104,PH21-015,50768.0,Pronghorn,Male,2021-12-01,Book Cliffs,Danish Flat,Detected
4,BCP1105,PH21-004,50774.0,Pronghorn,Female,2021-12-01,Book Cliffs,Danish Flat,Detected


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 113 entries, 0 to 112
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   sample_id          113 non-null    object        
 1   archive_id         15 non-null     object        
 2   collar_id          83 non-null     float64       
 3   species            108 non-null    object        
 4   sex                83 non-null     object        
 5   capture_date       108 non-null    datetime64[ns]
 6   capture_unit       108 non-null    object        
 7   staging_area       50 non-null     object        
 8   bluetongue_result  110 non-null    object        
dtypes: datetime64[ns](1), float64(1), object(7)
memory usage: 8.1+ KB


In [9]:
display(sheep_goat_df.head())
sheep_goat_df.info()

,sample_id,collar_id,species,sex,capture_date,capture_unit,staging_area,comments,movi_elisa_val,movi_elisa_result,movi_pcr_result,lentivirus_val,lentivirus_result,ehdv_val,ehdv_result,bluetongue_result,leukotoxin_lktA_result,tonsular_culture_result,tonsular_culture_isolate
0,SRS1101,50750.0,Bighorn Sheep,Female,2021-12-03,North San Rafael,Swinging Bridge,2 tonsillar swabs and 2 nasal swabs (SRS1101),80.903,Detected,Not detected,1.366,Negative,103.562,Negative,Negative,Not detected,No Pasteurella isolated.,None
1,SRS1102,50744.0,Bighorn Sheep,Female,2021-12-04,North San Rafael,Swinging Bridge,2 tonsillar swabs and 2 nasal swabs (SRS1102),61.281,Detected,Not detected,0.936,Negative,39.983,Indeterminate,Positive,Not detected,No Pasteurella isolated.,None
2,SRS1103,50749.0,Bighorn Sheep,Female,2021-12-03,North San Rafael,Swinging Bridge,"Short blood, veins flat, worried about heat. 2...",90.380,Detected,Detected,3.908,Negative,65.718,Negative,Positive,Not detected,No Pasteurella isolated.,None
3,SRS1104,50758.0,Bighorn Sheep,Female,2021-12-04,North San Rafael,Swinging Bridge,2 tonsillar swabs and 2 nasal swabs (SRS1104),72.088,Detected,None,2.541,Negative,61.098,Negative,Positive,None,None,None
4,SRS1105,50752.0,Bighorn Sheep,Female,2021-12-03,North San Rafael,Swinging Bridge,"Recapture. Old collar: 75535, 148.307. 2 tonsi...",74.081,Detected,Not detected,5.923,Negative,78.161,Negative,Negative,Not detected,No Pasteurella isolated.,None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77 entries, 0 to 76
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   sample_id                 77 non-null     object        
 1   collar_id                 62 non-null     float64       
 2   species                   68 non-null     object        
 3   sex                       64 non-null     object        
 4   capture_date              66 non-null     datetime64[ns]
 5   capture_unit              66 non-null     object        
 6   staging_area              10 non-null     object        
 7   comments                  19 non-null     object        
 8   movi_elisa_val            77 non-null     float64       
 9   movi_elisa_result         77 non-null     object        
 10  movi_pcr_result           65 non-null     object        
 11  lentivirus_val            77 non-null     float64       
 12  lentivirus_result       

In [10]:
#concat dfs
species_dfs = [bison_df, deer_df, elk_df, pronghorn_df, sheep_goat_df]
df = pd.concat(species_dfs)


### Drop unnecessary columns and rows

Many of the lab results in the database are not relevant to this project. For example, if the animal is pregnant, the exact value of the pregnancy test does not matter here. And as noted in the *definitions.md* file, columns like 'archive_id' are not useful for this project. 

There are also a handful of entries that do not have any associated lab results, as noted by the DWR Office's comments in the original spreadsheets. These rows are dropped. 

In [11]:
#drop columns not relevant to this project
df = df.drop(columns=['archive_id', 'preg_val', 'collar_id', 'ehdv_type1_val', 'ehdv_type2_val', 'ehdv_type6_val', 'movi_elisa_val', 'lentivirus_val', 'ehdv_val', 'tonsular_culture_isolate'])

#drop known duplicates and animals with missing samples.
comments_to_drop = ['*Double number, one sample', '*No record', '*missing sample']
df = df.drop(df[df.comments.isin(comments_to_drop)].index)

As a means of inspecting the data, let's look at the various values contained within each column:

In [12]:
for col in df.columns:
    print('The {} column has the following value counts: \n {} \n'.format(col, df[col].value_counts()))

The sample_id column has the following value counts: 
 sample_id
WM1101      16
WM1109      16
WM1103      16
WM1104      16
WM1105      16
            ..
OS1102       1
OS1103       1
OS1104       1
OS1105       1
BH21-010     1
Name: count, Length: 672, dtype: int64 

The species column has the following value counts: 
 species
Mule Deer        452
Elk              142
Pronghorn        104
Mt Goat           51
Bison             32
Bighorn Sheep     14
Name: count, dtype: int64 

The sex column has the following value counts: 
 sex
Female    685
Male       81
Name: count, dtype: int64 

The capture_date column has the following value counts: 
 capture_date
2021-12-11    93
2021-12-05    87
2022-02-06    31
2021-12-20    31
2021-12-10    30
2021-12-18    29
2021-12-01    29
2022-02-05    26
2022-02-24    25
2021-12-13    25
2022-02-28    25
2021-11-23    24
2021-12-12    24
2021-11-29    24
2021-12-16    23
2021-11-30    23
2021-11-22    22
2021-12-14    20
2021-11-21    18
2022-02-27 

Two things jump out: 
1. There should not be more than one instance of any given sample_id. 
    - Repeated sample_ids appear in the original data provided by the DWR; the reasons for this duplication are under investigation. For now, I will drop the duplicated ids.
2. The columns that report bovine viral diarrhea and lentivirus are entirely negative, meaning those diseases are abscent from the database. We can drop those columns entirely. 

In [13]:
#drop duplicate sample_ids
df = df.drop_duplicates(subset='sample_id')

#drop variables with all-negative results
df = df.drop(columns=['bvd_type1_result', 'bvd_type2_result', 'bvd_result', 'lentivirus_result'])

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 672 entries, 0 to 76
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   sample_id                672 non-null    object        
 1   species                  650 non-null    object        
 2   sex                      621 non-null    object        
 3   capture_date             648 non-null    datetime64[ns]
 4   capture_unit             648 non-null    object        
 5   ehdv_result              419 non-null    object        
 6   preg_result              170 non-null    object        
 7   bluetongue_result        660 non-null    object        
 8   adenovirus_result        320 non-null    object        
 9   staging_area             188 non-null    object        
 10  age                      140 non-null    float64       
 11  comments                 16 non-null     object        
 12  ehdv_type1_result        140 non-null    o

### Data Types

Before addressing the structural concerns, let's clean up the data types. To ease analysis later on, variables are encoded as binary values wherever possible (e.g., 'sex' with values of 'Female' and 'Male' becomes 'female' with values of 1 or 0). 

In [22]:
#encode sex as boolean 'female'
df['female'] = df.sex.apply(lambda row: 1 if row == 'Female' else 0)
del df['sex']

#set test results to 1/0
df.ehdv_result = df.ehdv_result.apply(lambda row: 1 if row in ['POSITIVE', 'Weak Positive', 'Positive'] else 0)
df.preg_result = df.preg_result(lambda row: 1 if row == 'Pregnant' else 0)
df.bluetongue_result = df.bluetongue_result.apply(lambda row: 1 if row in ['Detected', 'Positive'] else 0)



## ML Modeling

Jumping ahead in the process, I want to see about predicting test results given the available data. Since bluetongue is the only test done on all species, I'll simplify to look at that disease. 

In [15]:
#make df
bluetongue_df = df[['sample_id', 'species', 'sex', 'capture_date', 'capture_unit', 'bluetongue_result']].set_index('sample_id')

#drop date
bluetongue_df = bluetongue_df.drop(columns='capture_date')

#drop missing values
bluetongue_df = bluetongue_df.dropna()

In [16]:
bluetongue_df = pd.get_dummies(bluetongue_df, columns=['species', 'sex', 'capture_unit'], drop_first=True)

In [17]:
#split into X,y
X, y = bluetongue_df.drop(columns=['bluetongue_result']), bluetongue_df.bluetongue_result

#train test split
X_train, X_test, y_train, y_test = train_test_split(X, y)

#instantiate
model = LogisticRegression()

#fit
model.fit(X_train, y_train)

#predict
y_pred = model.predict(X_test)


In [19]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.95      0.87       114
           1       0.74      0.40      0.52        42

    accuracy                           0.80       156
   macro avg       0.78      0.68      0.70       156
weighted avg       0.79      0.80      0.78       156

